In [54]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

from dummies_bins_test_train_cv import get_Xy_train_test, cross_validation_process
# from clean_chess_game_log import main_cleanup
# _,_,_ = main_cleanup('../data/dest.pgn')

In [55]:
df = pd.read_csv('../data/use_for_predictions.csv')

In [57]:
X_train, X_test, y_train, y_test, X, y, df_clean = get_Xy_train_test(df, .985, .995)

y Shape: (2098,)
X Shape: (2098, 30)
X_train Shape: (2077, 30)
X_test Shape: (21, 30)
y_train Shape: (2077,)
y_test Shape: (21,)


In [58]:
df_clean.columns

Index(['color', 'diff_bin', 'gt_300', 'gt_600', 'st_1', 'st_2', 'st_6', 'st_7',
       'st_8', 'st_9', 'st_10', 'st_11', 'st_12', 'st_13', 'st_14', 'st_15',
       'st_16', 'st_17', 'st_18', 'st_19', 'st_20', 'st_21', 'st_22', 'st_23',
       'wd_1', 'wd_2', 'wd_3', 'wd_4', 'wd_5', 'wd_6'],
      dtype='object')

In [59]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

In [60]:
X_train.shape

(2077, 30)

In [61]:
X_test.shape

(21, 30)

In [62]:
y_train.shape

(2077,)

In [63]:
# std_sclr = StandardScaler()
# X_train = std_sclr.fit_transform(X_train)
# X_test = std_sclr.fit_transform(X_test)

In [64]:
# std_sclr.scale_

In [97]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=64, kernel_initializer='uniform',
                         activation='relu', input_dim=30))

    classifier.add(
        Dense(units=256, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='relu'))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [98]:
classifier = _classifier()

In [ ]:
classifier.fit(X_train, y_train, batch_size=8, epochs=30,
               class_weight='balanced')

Epoch 1/30
2077/2077 [==============================] - 2s 1ms/step - loss: 0.6759 - acc: 0.5826
Epoch 2/30
2077/2077 [==============================] - 1s 583us/step - loss: 0.6221 - acc: 0.6567
Epoch 3/30
2077/2077 [==============================] - 1s 646us/step - loss: 0.6276 - acc: 0.6216
Epoch 4/30
2077/2077 [==============================] - 1s 576us/step - loss: 0.5982 - acc: 0.6745
Epoch 5/30
2077/2077 [==============================] - 1s 642us/step - loss: 0.5904 - acc: 0.6895
Epoch 6/30
2077/2077 [==============================] - 1s 604us/step - loss: 0.5827 - acc: 0.6962
Epoch 7/30
1448/2077 [===================>..........] - ETA: 0s - loss: 0.5830 - acc: 0.6954

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [96]:
print(f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')

71.4%


In [ ]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=64, kernel_initializer='uniform',
                         activation='relu', input_dim=30,
                         kernel_regularizer=regularizers.l2(0.01),
                         activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.16))

    classifier.add(
        Dense(units=256, kernel_initializer='uniform', activation='softmax',
              kernel_regularizer=regularizers.l2(0.01),
              activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.08))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='relu'))

    classifier.add(Dropout(rate=0.04))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='softmax',
              kernel_regularizer=regularizers.l2(0.01),
              activity_regularizer=regularizers.l1(0.01)))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [ ]:
bs_1 = randint(10,20)
bs_2 = randint(20,30)
ep_1 = randint(100,200)
ep_2 = randint(200,300)

parameters = {'batch_size': [bs_1, bs_2],
              'nb_epoch': [ep_1, ep_2],
              'optimizer': ['Nadam', 'Adamax']}

In [ ]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [ ]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='f1_micro', cv=5, n_jobs=-4)

In [ ]:
print([bs_1, bs_2, ep_1, ep_2])
grid_search = grid_search.fit(X=X_train, y=y_train)

In [ ]:
best_params = grid_search.best_params_
best_params

In [ ]:
best_accu = grid_search.best_score_
best_accu

In [ ]:
grid_search.cv_results_

In [ ]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'classification_report:\n{classification_report(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'average_precision_score:\n{average_precision_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'f1_score:\n{f1_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'accuracy_score:\n{accuracy_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'cohen_kappa_score:\n{cohen_kappa_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_curve:\n{roc_curve(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_auc_score:\n{roc_auc_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'brier_score_loss:\n{brier_score_loss(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'recall_score:\n{recall_score(y_test, y_pred)*100}%')

In [ ]:
print(f'Accuracy = {round((cm[0][0]+cm[1][1])/(cm.sum())/.01,1)}%')

In [ ]:
def dropout_classifier_set():
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
# classifier = dropout_classifier()

In [ ]:
classifier.fit(X_train, y_train, batch_size=13, 
                   epochs=250, shuffle=False)

In [ ]:
batch_list = [134, 67, 28, 14, 7, 4]
for num in batch_list:
    classifier.fit(X_train, y_train, batch_size=num, 
                   epochs=100, shuffle=False)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'Accuracy = {(cm[0][0]+cm[1][1])/(cm.sum())/.01}%')

### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [ ]:
classifier = KerasClassifier(build_fn=_classifier, batch_size=12, epochs=200)

In [ ]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-2, scoring='accuracy')
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [ ]:
accus

In [ ]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier